# eQTM Analysis
## Author: Peter Allen

<style type="text/css">

h1.title {
  font-size: 38px;
  color: DarkRed;
  text-align: center;
}
h4.author { /* Header 4 - and the author and data headers use this too  */
    font-size: 18px;
  font-family: "Times New Roman", Times, serif;
  color: DarkRed;
  text-align: center;
}

h1, h2, h3 {
  text-align: center;
}
</style>

## Importing necessary Libraries

In [ ]:
library(data.table)
library(readxl)
library(dplyr)
library(Haplin)
library(IlluminaHumanMethylationEPICanno.ilm10b2.hg19)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
library(GenomicFeatures)
library(edgeR)
library(kableExtra)

## Importing metadata and filtering by which have both RNA-seq & Methylation Data


In [ ]:
meta <- data.frame(read_excel("rna-seq_analysis/data/RNA_SampleData.xlsx", col_types = c("skip", "text", "numeric", "skip", "skip", "text", "numeric", "text", "skip", "text", "skip"), na = c(".", "N/A")))
meta <- meta[meta$Methyl.Array.data.ID %like% "PSR",]
rownames(meta) <- meta[,1]
meta <- meta[,-1]
rownames(meta) <- gsub("-",".", rownames(meta))
meta$SSc.subset..lim.diff..or.sine[which(is.na(meta$SSc.subset..lim.diff..or.sine))] <- "Healthy"
meta <- meta %>% filter(!is.na(RNA.seq.data.ID))

## Importing Methylation Data

Importing the methylation data. There are two types of beta files, imputed and nonimputed. The imputed data will be used for calculating principal components which will be used to regress out potential technical variance downstream.

In [ ]:
## Imputed Betas
finalBetas_auto <- data.frame(fread("methylation_analysis/data/finalBetas_autos_Imp.txt", stringsAsFactors = FALSE))
rownames(finalBetas_auto) <- finalBetas_auto[,1]
finalBetas_auto <- finalBetas_auto[,-1]
colnames(finalBetas_auto)<-gsub("X","", colnames(finalBetas_auto))
finalBetas_auto <- finalBetas_auto[, order(colnames(finalBetas_auto))]

finalBetas_x <- data.frame(fread("methylation_analysis/data/finalBetas_X_Imp.txt", stringsAsFactors = FALSE))
rownames(finalBetas_x) <- finalBetas_x[,1]
finalBetas_x <- finalBetas_x[,-1]
colnames(finalBetas_x)<-gsub("X","", colnames(finalBetas_x))
finalBetas_x <- finalBetas_x[, order(colnames(finalBetas_x))]

finalBetasImp <- rbind(finalBetas_auto, finalBetas_x)

rm(finalBetas_auto, finalBetas_x)

## Non-imputed Betas
finalBetas_auto.nonimp <- data.frame(fread("methylation_analysis/data/finalBetas_autos_NonImp.txt", stringsAsFactors = FALSE))
rownames(finalBetas_auto.nonimp) <- finalBetas_auto.nonimp[,1]
finalBetas_auto.nonimp <- finalBetas_auto.nonimp[,-1]
colnames(finalBetas_auto.nonimp)<-gsub("X","", colnames(finalBetas_auto.nonimp))
finalBetas_auto.nonimp <- finalBetas_auto.nonimp[, order(colnames(finalBetas_auto.nonimp))]

finalBetas_x.nonimp <- data.frame(fread("methylation_analysis/data/finalBetas_X_NonImp.txt", stringsAsFactors = FALSE))
rownames(finalBetas_x.nonimp) <- finalBetas_x.nonimp[,1]
finalBetas_x.nonimp <- finalBetas_x.nonimp[,-1]
colnames(finalBetas_x.nonimp)<-gsub("X","", colnames(finalBetas_x.nonimp))
finalBetas_x.nonimp <- finalBetas_x.nonimp[, order(colnames(finalBetas_x.nonimp))]

finalBetasNonImp <- rbind(finalBetas_auto.nonimp, finalBetas_x.nonimp)
rm(finalBetas_auto.nonimp, finalBetas_x.nonimp)

finalBetasNonImp <- finalBetasNonImp[,match(rownames(meta), colnames(finalBetasNonImp))]
finalBetasImp <- finalBetasImp[,match(rownames(meta), colnames(finalBetasImp))]

## Performing Principal Component Analysis

In [ ]:
pca <- prcomp(t(na.omit(finalBetasImp)))$x

## Writing function to regress out the effects of the PCs from the beta values

In [ ]:
Beta_adjusted <- function(x) {
  tmp <- lm(as.numeric(x) ~ pca[,1] + pca[,2], na.action=na.exclude)
  beta_tmp <- tmp$residuals + tmp$coefficients[1]
  names(beta_tmp)<-colnames(finalBetasNonImp)[as.numeric(names(beta_tmp))]
  beta_tmp <- beta_tmp[colnames(finalBetasNonImp)]
  names(beta_tmp)<-colnames(finalBetasNonImp)
  return(beta_tmp)
}

adjusted_betas <- apply(na.omit(finalBetasNonImp), 1, Beta_adjusted)
adjusted_betas <- data.frame(adjusted_betas)

## Importing RNA-seq Data

Now that the methylation data has been processed, the RNA-seq counts data will be imported to start building our model

In [ ]:
## Import the RNA-seq data
expression_data <- read.delim("rna-seq_analysis/data/GSE196070_raw_counts_matrix.txt")
colnames(expression_data)<-gsub("X","", colnames(expression_data))

expression_data$symbol <- make.unique(as.character(expression_data$symbol), sep="_")

rownames(expression_data) <- expression_data$symbol
expression_data <- expression_data[,-1]

expression_data_meta <- expression_data[,colnames(expression_data) %in% meta$RNA.seq.data.ID]
colnames(expression_data_meta) <- rownames(meta)[match(colnames(expression_data_meta), meta$RNA.seq.data.ID)]
expression_data_meta <- expression_data_meta[,order(colnames(expression_data_meta))]

expression_data <- cbind(expression_data[,1:3], expression_data_meta)
meta_rna <- meta[na.omit(match(colnames(expression_data), rownames(meta))),]

## Filtering and Processing the Counts Data as was performed in the RNA-seq analysis

In [ ]:
group <- factor(meta_rna$SLE.SSc)
y <- DGEList(counts=expression_data[,4:ncol(expression_data)],group=group)
keep <- filterByExpr(y)
y <- y[keep,,keep.lib.sizes=FALSE]
y <- calcNormFactors(y, method = "TMM")
tmm <- cpm(y)

## Annotating the Methylation Data for the top CpGs (FDR < 0.45)

In [ ]:
na.omit(match(top.associated.cpgs$CpG,colnames(adjusted_betas))) %>% head()

top.associated.cpgs %>% head()

In [ ]:
top.associated.cpgs <- data.frame(fread("methylation_analysis/output/methylated_genes.txt", stringsAsFactors = FALSE))

top.associated.cpgs <- top.associated.cpgs %>% filter(p.adjusted < 0.45)
top.associated.betas <- adjusted_betas[,match(top.associated.cpgs$CpG,colnames(adjusted_betas))]

EPIC <- getAnnotation(IlluminaHumanMethylationEPICanno.ilm10b2.hg19)

EPIC <- as.data.frame(EPIC) %>% filter(rownames(.) %in% top.associated.cpgs$CpG) %>% arrange(match(rownames(.), top.associated.cpgs$CpG))
EPIC  <- as_tibble(EPIC)

EPIC.granges <- data.frame(EPIC[,1:2])
EPIC.granges$stop <- EPIC.granges$pos+5
EPIC.granges$CpG <- EPIC$Name

colnames(EPIC.granges) <- c("chr", "start", "stop", "CpG")

genes <- annotateTranscripts(TxDb.Hsapiens.UCSC.hg19.knownGene, annotationPackage = "org.Hs.eg.db")
cpgs <- makeGRangesFromDataFrame(EPIC.granges)


epic.genes <- matchGenes(cpgs,genes)

EPIC$GencodeBasicV12_NAME <- epic.genes$name
cpgs$Gene <- epic.genes$name
cpgs$CpG <- EPIC.granges$CpG

EPIC <- EPIC %>% dplyr::select(chr, pos, Relation_to_Island, GencodeBasicV12_NAME, Name)

## Annotating the top RNA-seq Analysis Transcripts (FDR < 0.4)

In [ ]:
##-- Annotating Gene Expression Data for regression
countMeta <- read.delim("top_1500_gene_regions_fdr.txt")
rownames(countMeta) <- countMeta[,1]
countMeta[,1] <- NULL

countMeta <- as.data.frame(countMeta) %>% filter(FDR < 0.4)

##-- Subsetting the data to top genes
countData.subset <- as.data.frame(tmm) %>% filter(rownames(.) %in% rownames(countMeta)) %>% arrange(match(rownames(.), rownames(countMeta)))

# Matching meta to count data
countData.meta <- countMeta %>% filter(rownames(.) %in% rownames(countData.subset)) %>% arrange(match(rownames(.), rownames(countData.subset)))

## For each of the transcripts, find all the CpG's 100kb up and downstream of promoter site

In [ ]:
## Making Granges object from the RNA metasheet
rna.granges <- data.frame(chr=countData.meta$Chr, start=countData.meta$Start-100000, stop=countData.meta$End+100000)

rna.granges.list <- list()
rna.transcripts <- list()

for (i in 1:nrow(rna.granges)){
   rna.granges.list[[i]] <- rna.granges[i,] 
}

for (i in 1:length(rna.granges.list)){
    rna.transcript=makeGRangesFromDataFrame(rna.granges.list[i])
    overlaps <- subsetByOverlaps(cpgs, rna.transcript)
    rna.transcripts[[i]] <- data.frame(overlaps) # %>% distinct(Gene, .keep_all = TRUE)
    rna.transcripts$CpG <- cpgs$CpG[match(rna.transcripts[[i]]$Gene, cpgs$Gene)]
}


## Performing the eQTM Analysis

To perform the eQTM analysis, a linear regression model was used with the transcript score being the outcome with methylation score (beta value) being considered a fixed effect along with age. 

In [ ]:
beta.values <- list()
cpg.regression.values <- list()
regression.df <- data.frame()

for (i in 1:length(rna.transcripts)){
  for (j in 1:nrow(as.data.frame(rna.transcripts[i]))){
    tmp <- as.data.frame(rna.transcripts[i])
    
    if(nrow(tmp) != 0){
      methylation.data <- top.associated.betas[,na.omit(match(tmp$CpG[j], colnames(top.associated.betas)))]           
    } else{
      methylation.data <- data.frame(0)
    }
    
    
    if(length(methylation.data) > 1){
      regression <- lm(as.numeric(countData.subset[i,]) ~ methylation.data + as.numeric(meta_rna$Age.at.enrollment))
      coefficient <- data.frame(summary(regression)$coeff[2,1])
      pval  <- data.frame(summary(regression)$coeff[2,4])
      transcript <- rownames(countData.subset)[i]
      
      regression.value <- cbind(transcript, coefficient, pval)
      
    } else{
      regression.value = data.frame(0)
    }
    
    if(j==1 && regression.value != 0){
      regression.df <- data.frame()
      names(regression.value) <- c("transcript","coefficient", "pval")
      regression.df <- rbind(regression.df, regression.value)
    } else if (j > 1 && regression.value != 0){
      names(regression.value) <- c("transcript","coefficient", "pval")
      regression.df <- rbind(regression.df, regression.value)
    }
    
    if(j==nrow(as.data.frame(rna.transcripts[i])) && j>0 && length(as.data.frame(rna.transcripts[i])) > 1){
      regression.df <- cbind(as.data.frame(rna.transcripts[i])$Gene, as.data.frame(rna.transcripts[i])$CpG, as.data.frame(rna.transcripts[i])$seqnames, as.data.frame(rna.transcripts[i])$start, regression.df)
      colnames(regression.df) <- c("CpG_Gene", "CpG", "Chr", "Position", "Transcript","coefficient", "pval")
      cpg.regression.values[[i]] <- regression.df %>% arrange(pval)
      names(cpg.regression.values[[i]]) <- c("CpG_Gene", "CpG", "CpG_Chr", "Position", "Transcript", "coefficient", "pval")
    } else{
      cpg.regression.values[[i]] <- data.frame(0)
    }
  }
}

cpg.regression.values <- cpg.regression.values[lapply(cpg.regression.values,length)>1]

total.cpg.regression.values <- cpg.regression.values


eQTM_results_FDR <- rbindlist(total.cpg.regression.values, use.names=FALSE, fill=FALSE, idcol=NULL) %>% arrange(pval)

eQTM_results_FDR 

## Second model using expanding number of CpGs to any below 99% FDR (148 CpGs)
This allows for more CpGs to be analyzed should any have biological relevance but not considered previously due to not meeting the initial FDR threshold.

In [ ]:
top.associated.cpgs <- data.frame(fread("methylation_analysis/output/methylated_genes.txt", stringsAsFactors = FALSE))

top.associated.cpgs <- top.associated.cpgs %>% filter(p.adjusted < 0.99)
top.associated.betas <- adjusted_betas[,match(top.associated.cpgs$CpG,colnames(adjusted_betas))]

EPIC <- getAnnotation(IlluminaHumanMethylationEPICanno.ilm10b2.hg19)

EPIC <- as.data.frame(EPIC) %>% filter(rownames(.) %in% top.associated.cpgs$CpG) %>% arrange(match(rownames(.), top.associated.cpgs$CpG))
EPIC  <- as_tibble(EPIC)

EPIC.granges <- data.frame(EPIC[,1:2])
EPIC.granges$stop <- EPIC.granges$pos+5
EPIC.granges$CpG <- EPIC$Name

colnames(EPIC.granges) <- c("chr", "start", "stop", "CpG")

genes <- annotateTranscripts(TxDb.Hsapiens.UCSC.hg19.knownGene)
cpgs <- makeGRangesFromDataFrame(EPIC.granges)


epic.genes <- matchGenes(cpgs,genes)

EPIC$GencodeBasicV12_NAME <- epic.genes$name
cpgs$Gene <- epic.genes$name
cpgs$CpG <- EPIC.granges$CpG

EPIC <- EPIC %>% dplyr::select(chr, pos, Relation_to_Island, GencodeBasicV12_NAME, Name)

## Annotating the top RNA-seq Analysis Transcripts (FDR < 0.4)

In [ ]:
##-- Annotating Gene Expression Data for regression
countMeta <- read.delim("top_1500_gene_regions_fdr.txt")
rownames(countMeta) <- countMeta[,1]
countMeta[,1] <- NULL

countMeta <- as.data.frame(countMeta) %>% filter(FDR < 0.4)

##-- Subsetting the data to top genes
countData.subset <- as.data.frame(tmm) %>% filter(rownames(.) %in% rownames(countMeta)) %>% arrange(match(rownames(.), rownames(countMeta)))

# Matching meta to count data
countData.meta <- countMeta %>% filter(rownames(.) %in% rownames(countData.subset)) %>% arrange(match(rownames(.), rownames(countData.subset)))

## For each of the transcripts, find all the CpG's 100kb up and downstream of promoter site

In [ ]:
## Making Granges object from the RNA metasheet
rna.granges <- data.frame(chr=countData.meta$Chr, start=countData.meta$Start-100000, stop=countData.meta$End+100000)

rna.granges.list <- list()
rna.transcripts <- list()

for (i in 1:nrow(rna.granges)){
   rna.granges.list[[i]] <- rna.granges[i,] 
}

for (i in 1:length(rna.granges.list)){
    rna.transcript=makeGRangesFromDataFrame(rna.granges.list[i])
    overlaps <- subsetByOverlaps(cpgs, rna.transcript)
    rna.transcripts[[i]] <- data.frame(overlaps) # %>% distinct(Gene, .keep_all = TRUE)
    rna.transcripts$CpG <- cpgs$CpG[match(rna.transcripts[[i]]$Gene, cpgs$Gene)]
}


## Performing the eQTM Analysis

To perform the eQTM analysis, a linear regression model was used with the transcript score being the outcome with methylation score (beta value) being considered a fixed effect along with age. 

In [ ]:
beta.values <- list()
cpg.regression.values <- list()
regression.df <- data.frame()

for (i in 1:length(rna.transcripts)){
  for (j in 1:nrow(as.data.frame(rna.transcripts[i]))){
    tmp <- as.data.frame(rna.transcripts[i])
    
    if(nrow(tmp) != 0){
      methylation.data <- top.associated.betas[,na.omit(match(tmp$CpG[j], colnames(top.associated.betas)))]           
    } else{
      methylation.data <- data.frame(0)
    }
    
    
    if(length(methylation.data) > 1){
      regression <- lm(as.numeric(countData.subset[i,]) ~ methylation.data + as.numeric(meta_rna$Age.at.enrollment))
      coefficient <- data.frame(summary(regression)$coeff[2,1])
      pval  <- data.frame(summary(regression)$coeff[2,4])
      transcript <- rownames(countData.subset)[i]
      
      regression.value <- cbind(transcript, coefficient, pval)
      
    } else{
      regression.value = data.frame(0)
    }
    
    if(j==1 && regression.value != 0){
      regression.df <- data.frame()
      names(regression.value) <- c("transcript","coefficient", "pval")
      regression.df <- rbind(regression.df, regression.value)
    } else if (j > 1 && regression.value != 0){
      names(regression.value) <- c("transcript","coefficient", "pval")
      regression.df <- rbind(regression.df, regression.value)
    }
    
    if(j==nrow(as.data.frame(rna.transcripts[i])) && j>0 && length(as.data.frame(rna.transcripts[i])) > 1){
      regression.df <- cbind(as.data.frame(rna.transcripts[i])$Gene, as.data.frame(rna.transcripts[i])$CpG, as.data.frame(rna.transcripts[i])$seqnames, as.data.frame(rna.transcripts[i])$start, regression.df)
      colnames(regression.df) <- c("CpG_Gene", "CpG", "Chr", "Position", "Transcript","coefficient", "pval")
      cpg.regression.values[[i]] <- regression.df %>% arrange(pval)
      names(cpg.regression.values[[i]]) <- c("CpG_Gene", "CpG", "CpG_Chr", "Position", "Transcript", "coefficient", "pval")
    } else{
      cpg.regression.values[[i]] <- data.frame(0)
    }
  }
}

cpg.regression.values <- cpg.regression.values[lapply(cpg.regression.values,length)>1]

total.cpg.regression.values <- cpg.regression.values


eQTM_results_expanded <- rbindlist(total.cpg.regression.values, use.names=FALSE, fill=FALSE, idcol=NULL) %>% arrange(pval)

eQTM_results_expanded 